<a href="https://colab.research.google.com/github/Samiimasmoudii/Deep-Learning-Course/blob/main/Optimisation_Wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Optimisation des architectures et des hyperparamètres**
Optimisation d'un CNN

In [ ]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Liste des packages à installer
packages = ["tensorflow", "scikit-learn"]

for package in packages:
    install_package(package)

print("Installation terminée.")


Installation terminée.


In [ ]:
import tensorflow as tf
import sklearn

print("TensorFlow version:", tf.__version__)
print("scikit-learn version:", sklearn.__version__)


TensorFlow version: 2.17.1
scikit-learn version: 1.5.2


Optimisation par **GridSearch**

Importer les bibliothèques nécessaires

In [ ]:
!pip install scikeras
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV


**Étape 1** : Préparer les données Utilisons le dataset CIFAR-10.

In [ ]:


# Charger les données
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normaliser les données
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convertir les labels en encodage one-hot
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


**Étape 2** : Construire le modèle de base

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model(learning_rate=0.01, dropout_rate=0.5, num_filters=32, kernel_size=3):
    model = Sequential()
    model.add(Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(32, 32, 3))) # IMage size is 32*32 with 3 colors
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout_rate)) # Dropout Rate = 0.5
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


**Étape 3** : Définir la grille d'hyperparamètres

In [ ]:
from sklearn.model_selection import GridSearchCV


# Créer le wrapper KerasClassifier
model = KerasClassifier(model=create_model, epochs=10, batch_size=64, verbose=0)

# Définir la grille d'hyperparamètres
param_grid = {
    'model__learning_rate': [0.001, 0.01, 0.1],
    'model__dropout_rate': [0.3, 0.5, 0.7],
    'model__num_filters': [32, 64, 128],
    'model__kernel_size': [3, 5]
}

# Définir GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)


**Étape 4** : Effectuer la recherche de grille

In [ ]:
# Effectuer la recherche
grid_search_result = grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres : ", grid_search_result.best_params_)


Fitting 3 folds for each of 54 candidates, totalling 162 fits


**Étape 5** : Évaluer le modèle optimisé

In [ ]:
# Extraire les meilleurs hyperparamètres
best_params = grid_search_result.best_params_

# Créer et entraîner le modèle avec les meilleurs hyperparamètres
best_model = create_model(learning_rate=best_params['learning_rate'],
                          dropout_rate=best_params['dropout_rate'],
                          num_filters=best_params['num_filters'],
                          kernel_size=best_params['kernel_size'])

history = best_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Évaluer le modèle sur le jeu de test
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy sur le jeu de test : {test_accuracy * 100:.2f}%')


**Recherche des paramètres par RandomizedSearchCV**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Créer le wrapper KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=64, verbose=0)

# Définir la grille d'hyperparamètres
param_dist = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.3, 0.5, 0.7],
    'num_filters': [32, 64, 128],
    'kernel_size': [3, 5]
}

# Définir RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, verbose=1, n_jobs=-1)
# Effectuer la recherche
random_search_result = random_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres : ", random_search_result.best_params_)
# Extraire les meilleurs hyperparamètres
best_params = random_search_result.best_params_

# Créer et entraîner le modèle avec les meilleurs hyperparamètres
best_model = create_model(learning_rate=best_params['learning_rate'],
                          dropout_rate=best_params['dropout_rate'],
                          num_filters=best_params['num_filters'],
                          kernel_size=best_params['kernel_size'])

history = best_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Évaluer le modèle sur le jeu de test
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy sur le jeu de test : {test_accuracy * 100:.2f}%')


**Recherche des paramètres par optimisation bayésienne**

In [ ]:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

# Définir l'espace de recherche
space = {
    'learning_rate': hp.choice('learning_rate', [0.001, 0.01, 0.1]),
    'dropout_rate': hp.choice('dropout_rate', [0.3, 0.5, 0.7]),
    'num_filters': hp.choice('num_filters', [32, 64, 128]),
    'kernel_size': hp.choice('kernel_size', [3, 5])
}


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

def objective(params):
    model = create_model(params)

    # Split des données d'entraînement pour la validation
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2)

    # Entraîner le modèle
    history = model.fit(X_train_split, y_train_split, epochs=10, batch_size=64,
              validation_data=(X_val_split, y_val_split), verbose=0, callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

    # Évaluer le modèle
    val_loss, val_accuracy = model.evaluate(X_val_split, y_val_split, verbose=0)
    return {'loss': -val_accuracy, 'status': STATUS_OK}


In [ ]:
# Définir Trials pour stocker les résultats
trials = Trials()

# Lancer l'optimisation
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

print("Meilleurs hyperparamètres : ", best)


In [ ]:
# Traduire les hyperparamètres choisis par Hyperopt
best_params = {
    'learning_rate': [0.001, 0.01, 0.1][best['learning_rate']],
    'dropout_rate': [0.3, 0.5, 0.7][best['dropout_rate']],
    'num_filters': [32, 64, 128][best['num_filters']],
    'kernel_size': [3, 5][best['kernel_size']]
}

# Créer et entraîner le modèle avec les meilleurs hyperparamètres
best_model = create_model(best_params)
history = best_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Évaluer le modèle sur le jeu de test
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy sur le jeu de test : {test_accuracy * 100:.2f}%')
